Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Forecasting Pipeline - Automated ML

---

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/manymodels/03_Forecasting/03_Forecasting_Pipeline.png)

In this notebook we create a pipeline for doing batch forecasting of 11,973 AutoML models. The training and scoring of these models was completed in the Training notebook in this repository. We will set up the Pipeline for forecasting given the desired forecasting horizon. We utitlize the AutoMLPipelineBuilder to parallelize the process. For more information about the Data and Models refer to the Data Preparation and Training Notebooks. 

The pipeline set up is similar to the Training Pipeline in this repository. For more details on the steps and functions refer to the Training folder. 

### Prerequisites 
At this point, you should have already:

1. Created your AML Workspace using the [00_Setup_AML_Workspace notebook](../../00_Setup_AML_Workspace.ipynb)
2. Run [01_Data_Preparation.ipynb](../../01_Data_Preparation.ipynb) to create the dataset
3. Run [02_AutoML_Training_Pipeline.ipynb](../02_AutoML_Training_Pipeline/02_AutoML_Training_Pipeline.ipynb) to train the models

Install the azureml.contrib.automl.pipeline.steps package

In [ ]:
# !pip install azureml-contrib-automl-pipeline-steps

## 1.0 Call the Workspace, Datastore, and Compute

As we did in the Training Pipeline notebook, we need to call the Workspace. We also want to create variables for the datastore and compute cluster. 

### Connect to the workspace


In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd

# set up workspace
ws= Workspace.from_config() 

# Take a look at Workspace
ws.get_details()

# set up datastores
dstore = ws.get_default_datastore()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Default datastore name'] = dstore.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Attach existing compute resource


In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "cpucluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D16S_V3',
                                                           min_nodes=2,
                                                           max_nodes=20)
    # Create the cluster.
    compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
# For a more detailed view of current AmlCompute status, use get_status().

### Set up an Experiment

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'manymodels-forecasting-pipeline')

## 2.0 Call Registered FileDataset
In the Data Preparation notebook, we registered the orange juice inference data to the Workspace. You can choose to run the pipeline on the subet of 10 series or the full dataset of 11,973 series. We recommend starting with 10 series then expanding. 

In [ ]:
from azureml.core.dataset import Dataset

filedst_10_models = Dataset.get_by_name(ws, name='oj_data_small_inference')
filedst_10_models_input = filedst_10_models.as_named_input('forecast_10_models')
 
#filedst_all_models = Dataset.get_by_name(ws, name='oj_data_inference')
#filedst_all_models_input = filedst_all_models.as_named_input('forecast_all_models')

## 3.0 Build forecasting pipeline
Now that the data, models, and compute resources are set up, we can put together a pipeline for forecasting. 
### Set up the environment to run the script
Specify the conda dependencies for your script. This will allow us to install packages and configure the environment.

In [ ]:
training_experiment_name = "<training_experiment_name_goes_here>"
training_pipeline_run_id ="<training_pipeline_run_id_goes_here>"

### Create the configuration to wrap the entry script 
AutoMLPipelineBuilder is used to build the inference step for many models. You will need to determine the number of workers and nodes appropriate for your use case. The process_count_per_node is based off the number of cores of the compute VM. The node_count will determine the number of master nodes to use, increasing the node count will speed up the training process.

* <b>experiment</b>: Current experiment.

* <b>inference_data</b>: Inference dataset.

* <b>compute_target</b>: Compute target for inference.

* <b>node_count</b>: The number of compute nodes to be used for running the user script. We recommend to start with 3 and increase the node_count if the training time is taking too long.

* <b>process_count_per_node</b>: The number of processes per node.

* <b>run_invocation_timeout</b>: The run() method invocation timeout in seconds. The timeout should be set to maximum training time of one AutoML run(with some buffer), by default it's 60 seconds.

* <b>output_datastore</b>: Output datastore to output the inference results.

* <b>train_experiment_name</b>: Training experiment name where many models were trained.

* <b>train_run_id</b>: Training run id where many models were trained.

* <b>partition_column_names</b>: Partition column names.

* <b>time_column_name(Optional)</b>: Time column name if it is timeseries

* <b>target_column_name(Optional)</b>: Target column name if the inference dataset has the target column

<span style="color:red"><b>NOTE: There are limits on how many runs we can do in parallel per workspace, and we currently recommend to set the parallelism to maximum of 320 runs per experiment per workspace. If users want to have more parallelism and increase this limit they might encounter Too Many Requests errors (HTTP 429). </b></span>

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder

partition_column_names = ['Store', 'Brand']

inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(experiment=experiment,
                                                                              inference_data=filedst_10_models_input,
                                                                              compute_target=compute,
                                                                              node_count=2,
                                                                              process_count_per_node=8,
                                                                              run_invocation_timeout=300,
                                                                              output_datastore=dstore,
                                                                              train_experiment_name=training_experiment_name,
                                                                              train_run_id=training_pipeline_run_id,
                                                                              partition_column_names=partition_column_names,
                                                                              time_column_name="WeekStarting",
                                                                              target_column_name="Quantity")

## 4.0 Run the forecast pipeline
We can use the Experiment we created to track the runs of the pipeline and review the output.

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace = ws, steps=inference_steps)
run = experiment.submit(pipeline)

You can run the folowing command if you'd like to monitor the forecasting process in jupyter notebook. It will stream logs live while forecasting. 

**Note**: this command may not work for Notebook VM, however it should work on your local laptop.

In [ ]:
run.wait_for_completion(show_output=True)

Succesfully forecasted on AutoML Models.

## 5.0 Pipeline Outputs
The forecasting pipeline forecasts the orange juice quantity for a Store by Brand. The pipeline returns one file with the predictions for each store and outputs the result to the forecasting_output Blob container. The details of the blob container is listed in 'forecasting_output.txt' under Outputs+logs. 

The following code snippet:
1. Downloads the contents of the output folder that is passed in the parallel run step 
2. Reads the parallel_run_step.txt file that has the predictions as pandas dataframe and 
3. Displays the top 10 rows of the predictions

In [ ]:
import pandas as pd
import shutil
import os
import sys 
from scripts.helper import get_forecasting_output

forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"

forecast_file = get_forecasting_output(run, forecasting_results_name, forecasting_output_name)
df = pd.read_csv(forecast_file, delimiter=" ", header=None)
df.columns = ["Week Starting", "Store", "Brand", "Quantity",  "Advert", "Price" , "Revenue", "Predicted" ]
print("Prediction has ", df.shape[0], " rows. Here the first 10 rows are being displayed.")
df.head(10)

## 6.0 Publish and schedule the pipeline (Optional)

### 6.1 Publish the pipeline

Once you have a pipeline you're happy with, you can publish a pipeline so you can call it programmatically later on. See this [tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-your-first-pipeline#publish-a-pipeline) for additional information on publishing and calling pipelines.

In [ ]:
# published_pipeline = pipeline.publish(name = 'automl_forecast_many_models',
#                                      description = 'forecast many models',
#                                      version = '1',
#                                      continue_on_step_failure = False)

### 6.2 Schedule the pipeline
You can also [schedule the pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-schedule-pipelines) to run on a time-based or change-based schedule. This could be used to automatically retrain or forecast models every month or based on another trigger such as data drift.

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
# forecasting_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="automl_forecasting_recurring_schedule", 
#                             description="Schedule Forecasting Pipeline to run on the first day of every week",
#                             pipeline_id=forecasting_pipeline_id, 
#                             experiment_name=experiment.name, 
#                             recurrence=recurrence)